In [1]:
# install geocoder package

import sys
!{sys.executable} -m pip install geocoder
print('package installed')


     |████████████████████████████████| 102kB 4.8MB/s ta 0:00:011
     |████████████████████████████████| 92kB 5.1MB/s eta 0:00:011
     |████████████████████████████████| 829kB 10.6MB/s eta 0:00:01     |████████                        | 204kB 10.6MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
package installed


In [2]:
# install geopy package

!conda install -c conda-forge geopy --yes
print('package installed')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         393 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

In [3]:
# install folium package
!conda install -c conda-forge folium=0.5.0 --yes
print('package installed')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-2.9.2         |   py36h45558ae_0         613 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    pandas-1.0.5               |   py36h830a2c2_0        10.1 MB  conda-forge
    pysocks-1.7.1              |   py36h9f0ad1d_1          27 KB  conda-

In [4]:
#import libraries
import pandas as pd  # library for data analsysis
import requests  # library to handle requests
import geocoder
import json  # library to handle JSON files
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe
import numpy as np  # library to handle data in a vectorized manner
import matplotlib.cm as cm  # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values
from sklearn.cluster import KMeans  # import k-means from clustering stage
import folium  # map rendering library

In [5]:
# Get neighborhood data: 'cod_distrito', 'distrito', 'cod_barrio', 'barrio' 

neighbors = pd.read_csv('madrid-distritos-barrios.csv')

#clean dataframe and drop duplicates
neighbors = neighbors.drop(neighbors.columns[[4,5]], axis=1)
neighbors = neighbors.drop_duplicates(subset = ["distrito"])
neighbors = neighbors.astype(str)
neighbors['Code'] = neighbors['cod_distrito'] + neighbors['cod_barrio']
neighbors = neighbors.drop(['cod_distrito', 'cod_barrio'], axis=1)
neighbors.rename(columns={'distrito':'Borough', 'barrio':'Neighborhood'}, inplace=True)
neighbors = neighbors[['Code', 'Borough', 'Neighborhood']]
neighbors['Latitude'] = 0.0
neighbors['Longitude'] = 0.0

neighbors

,Code,Borough,Neighborhood,Latitude,Longitude
0,111,Centro,Palacio,0.0,0.0
6,221,Arganzuela,Imperial,0.0,0.0
13,331,Retiro,Pacífico,0.0,0.0
19,441,Salamanca,Recoletos,0.0,0.0
25,551,Chamartín,El Viso,0.0,0.0
31,661,Tetuán,Bellas Vistas,0.0,0.0
37,771,Chamberí,Gaztambide,0.0,0.0
43,881,Fuencarral-El Pardo,El Pardo,0.0,0.0
51,991,Moncloa-Aravaca,Casa de Campo,0.0,0.0
58,10101,Latina,Los Cármenes,0.0,0.0


In [6]:
# call for arcgis to assess the coordinates for each Neighborhood
for i in range(0, len(neighbors)):
	dir = neighbors.iloc[i,2] + ', ' + neighbors.iloc[i,1] + ', madrid, españa'  # "Neighborhood, Borough, 'madrid', 'españa'"
	g = geocoder.arcgis(dir)
	neighbors.iloc[i,3] = g.latlng[0]
	neighbors.iloc[i,4] = g.latlng[1]
    
neighbors

,Code,Borough,Neighborhood,Latitude,Longitude
0,111,Centro,Palacio,40.409630,-3.879790
6,221,Arganzuela,Imperial,40.400210,-3.696180
13,331,Retiro,Pacífico,40.401910,-3.676030
19,441,Salamanca,Recoletos,40.429720,-3.679750
25,551,Chamartín,El Viso,40.450000,-3.700000
31,661,Tetuán,Bellas Vistas,40.454570,-3.705520
37,771,Chamberí,Gaztambide,40.434900,-3.715510
43,881,Fuencarral-El Pardo,El Pardo,40.511644,-3.754898
51,991,Moncloa-Aravaca,Casa de Campo,40.435470,-3.731700
58,10101,Latina,Los Cármenes,40.402800,-3.731780


In [7]:
# Foursquare credentials
CLIENT_ID = 'XZQ0BJKCXSL1CH4MYIYXHE20YQYNEM1WFS4ZEBVPSKDQH4J5'
CLIENT_SECRET = '2JM1AYFVJ5XRXWRS4FCPP0RPB1XCINKSHUQGKBOI0P0A5KCO'
VERSION = '20180604'


In [8]:
# define the venues dataframe
venues_columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

madrid_venues = pd.DataFrame(columns=venues_columns)

In [9]:
# function finding venues for a given location
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [10]:
# call the function
madrid_venues = getNearbyVenues(names=neighbors['Neighborhood'],
                                   latitudes=neighbors['Latitude'],
                                   longitudes=neighbors['Longitude'])

madrid_venues

Palacio
Imperial


KeyError: 'groups'

In [ ]:
madrid_venues.groupby('Neighborhood').count()

In [ ]:
madrid_venues.head()

In [ ]:
madrid_venues.shape

In [ ]:
madrid_onehot = pd.get_dummies(madrid_venues[['Venue Category']], prefix="", prefix_sep="")
madrid_onehot

In [ ]:
madrid_onehot.insert(loc=0, column='Neighborhood', value=madrid_venues['Neighborhood'] )
madrid_onehot.shape

In [ ]:
madrid_onehot

In [ ]:
madrid_grouped = madrid_onehot.groupby('Neighborhood').mean().reset_index()
madrid_grouped.head()

In [ ]:
# function returning categories for each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [ ]:
# get top ten common venues for each Neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = madrid_grouped['Neighborhood']

for ind in np.arange(madrid_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(madrid_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# Clustering for Neighborhood

# set number of clusters
kclusters = 5

madrid_grouped_clustering = madrid_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(madrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

madrid_merged = neighbors


In [ ]:
# merge madrid_grouped with madrid_data to add latitude/longitude for each neighborhood
madrid_merged = madrid_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

madrid_merged.head()

In [ ]:
neighborhoods_venues_sorted.head()

In [ ]:
#searching location for 'madrid, españa'

address = 'madrid, españa'

geolocator = Nominatim(user_agent="mad_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid are {}, {}.'.format(latitude, longitude))



In [ ]:
# Map with clusters for 'Madrid, España'
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


In [ ]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [ ]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(madrid_merged['Latitude'], madrid_merged['Longitude'], madrid_merged['Neighborhood'], madrid_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

In [ ]:
map_clusters